인덱스

1. 목표
2. 탐색적 자료 분석 
3. 피처 엔지니어링 
4. 모델링
    1. 의사결정 나무
    2. 랜덤 포레스트
    3. XGBoost
5. 결론

# 1. 목표(Objective)

자동차 보험회사의 고객 LTV를 예측하는 것이 목표이다. LTV를 포함해 24개의 컬럼 데이터가 있으며, 종속변수인 LTV와 나머지 23개 컬럼(독립변수)과의 상관관계 분석을 통해 LTV를 예측하고자 한다. 

In [ ]:
import numpy as np # 넘파이 불러오기
import pandas as pd # 판다스 불러오기

import matplotlib.pyplot as plt #맷플랏립 불러오기
import seaborn as sns # 씨본 불러오기
%matplotlib inline
sns.set()
from subprocess import check_output

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 2. EDA (Exploratory Data Analysis)

In [ ]:
# 데이터 업로드
df = pd.read_csv('/kaggle/input/ibm-watson-marketing-customer-value-data/WA_Fn-UseC_-Marketing-Customer-Value-Analysis.csv')

In [ ]:
# 데이터프레임 보기
df.head(10)

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
# 데이터 개수와 빈값, 데이터 타입 등 확인
df.info()

In [ ]:
# int 데이터 분석
print('Income:', df['Income'].unique(), df['Income'].nunique())
print('Monthly Premium Auto:', df['Monthly Premium Auto'].unique(), df['Monthly Premium Auto'].nunique())
print('Months Since Last Claim:', df['Months Since Last Claim'].unique(), df['Months Since Last Claim'].nunique())
print('Months Since Policy Inception:', df['Months Since Policy Inception'].unique(), df['Months Since Policy Inception'].nunique())
print('Number of Open Complaints:', df['Number of Open Complaints'].unique(), df['Number of Open Complaints'].nunique())
print('Number of Policies:', df['Number of Policies'].unique(), df['Number of Policies'].nunique())

9134행 x 24열(종속변수1 + 독립변수 23)

16개의 범주형 데이터(categorical data)와 (값이 정해지지 않은) 8개의 연속형 데이터(continuous data)가 있고, 빈값은 없다.

범주형 데이터 - Customer, State, Response, Coverage, Effective To Date, Education, EmploymentStatus, Gender, Location Code, Marital Status, Policy Type, Policy, Renew Offer Type, Sales Channel, Vehicle Class, Vehicle Size  

연속형 데이터 - Customer Lifetime Value, Total Claim Amount, Income, Monthly Premium Auto, Months Since Last Claim, Months Since Policy Inception, Number of Open Complaints, Number of Policies

In [ ]:
# 각 컬럼의 유니크값 개수
df.nunique()

In [ ]:
# 숫자형 컬럼 통계 서머리
df.describe()

# (1) 연속형 데이터(Continuous Data) 분석

In [ ]:
# Customer LTV 분석
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.distplot(df['Customer Lifetime Value'])

plt.subplot(1, 2, 2)
sns.boxplot(df['Customer Lifetime Value'])

In [ ]:
# 사분범위(IQR, Interquartile Range) 구하기
Q1 = df['Customer Lifetime Value'].quantile(0.25)
Q3 = df['Customer Lifetime Value'].quantile(0.75)
IQR = Q3 - Q1
print(IQR)

# 최대값
print(Q3 + 1.5 * IQR)

# 사분범위로 아웃라이어 구하기
df[(df['Customer Lifetime Value'] > (Q3 + 1.5 * IQR))]['Customer Lifetime Value']

Q3에서 1.5 X IQR을 더한 값보다 벗어나는 위치에 있는 LTV 값이 817개 있다. 


In [ ]:
# Income 
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.distplot(df['Income']) # 소득의 디스트리뷰션

plt.subplot(1, 2, 2)
sns.scatterplot(df['Income'], df['Customer Lifetime Value']) #소득과 LTV의 관계

In [ ]:
# Monthly Premium Auto
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.distplot(df['Monthly Premium Auto']) # Monthly Premium Auto 분포

plt.subplot(1, 2, 2)
sns.scatterplot(df['Monthly Premium Auto'], df['Customer Lifetime Value']) # Monthly Premium Auto와 LTV의 관계

In [ ]:
# Months Since Last Claim
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.distplot(df['Months Since Last Claim']) #Months Since Last Claim 분포

plt.subplot(1, 2, 2)
sns.scatterplot(x='Months Since Last Claim', y='Customer Lifetime Value', data=df)  # Months Since Last Claim과 LTV의 관계

In [ ]:
# Months Since Policy Inception
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.distplot(df['Months Since Policy Inception']) # Months Since Policy Inception 분포

plt.subplot(1, 2, 2)
sns.scatterplot(x='Months Since Policy Inception', y='Customer Lifetime Value', data=df) # Months Since Policy Inception과 LTV 관계

In [ ]:
# Total Claim Amount
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.distplot(df['Total Claim Amount']) # Total Claim Amount 분포

plt.subplot(1, 2, 2)
sns.scatterplot(x='Total Claim Amount', y='Customer Lifetime Value', data=df) # Total Claim Amount과 LTV 관계

In [ ]:
# Number of Policies
plt.figure(figsize=(24,8))
plt.subplot(1, 3, 1)
sns.countplot(df['Number of Policies'])

plt.subplot(1, 3, 2)
sns.barplot(x='Number of Policies', y='Customer Lifetime Value', data=df)

In [ ]:
# Number of Open Complaints
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.countplot(df['Number of Open Complaints']) # Number of Open Complaints 항목별 개수

plt.subplot(1, 2, 2)
sns.barplot(x='Number of Open Complaints', y='Customer Lifetime Value', data=df) # Number of Open Complaints과 LTV 관계

In [ ]:
# .corr() 함수를 사용해 히트맵 그리기

plt.figure(figsize=(7,6))
sns.heatmap(df[['Customer Lifetime Value','Income','Monthly Premium Auto','Total Claim Amount', 'Months Since Last Claim', 'Months Since Policy Inception', 'Number of Policies', 'Number of Open Complaints']].corr(),annot = True)
plt.show()

Income, Months Since Last Clamin, Months Since Policy Inception, Number of Open Complaints는 LTV와 유의미한 상관관계가 보이지 않음

Monthly Premium Auto는 LTV와 선형적인 상관관계가 있고, Total Claim Amount도 어느 정도 상관관계를 가지고 있는 것으로 보임

Number of Policies의 경우 2개를 가진 경우 LTV가 가장 높았고 1개가 가장 낮았다. 나머지는 LTV가 비슷하다. 2개인 경우가 왜 3개 이상 보다 LTV가 가장 높은 지 보험증서 개수와 다른 변수(보험증서 유형, 오퍼타입 등)와 상관관계를 봐도 이렇다 할 이유를 찾기 힘들다. 


# (2) 범주형 데이터 (Categorical Data) 분석

In [ ]:
# Customer는 고객별 고유번호로 LTV와 상관관계가 없다고 생각해 분석하지 않음

In [ ]:
# State
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.countplot(df['State'])

plt.subplot(1, 2, 2)
sns.barplot(x='State', y='Customer Lifetime Value', data=df)

In [ ]:
# Response
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.countplot(df['Response'])

plt.subplot(1, 2, 2)
sns.barplot(x='Response', y='Customer Lifetime Value', data=df)

In [ ]:
# Coverage
plt.figure(figsize=(20,6))
plt.subplot(1, 2, 1)
sns.countplot(df['Coverage'])

plt.subplot(1, 2, 2)
sns.barplot(x='Coverage', y='Customer Lifetime Value', data=df)

In [ ]:
# Effective To Date 
df['Month'] = 0
df['Month'] = df['Effective To Date'].str.extract(r'(\d+)') #유효일 데이터에서 달(month)만 추출해 확인

print(df['Month'].unique())

모든 데이터의 계약 유효일은 2011년이고, 월 데이터만 추출해 본 결과 1월 또는 2월로 LTV와 상관관계가 없는 것으로 판단

In [ ]:
# Education
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.countplot(df['Education'])

plt.subplot(1, 2, 2)
sns.barplot(x='Education', y='Customer Lifetime Value', data=df)

In [ ]:
# EmploymentStatus
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.countplot(df['EmploymentStatus'])

plt.subplot(1, 2, 2)
sns.barplot(x='EmploymentStatus', y='Customer Lifetime Value', data=df)

In [ ]:
# Gender
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.countplot(df['Gender'])

plt.subplot(1, 2, 2)
sns.barplot(x='Gender', y='Customer Lifetime Value', data=df)

In [ ]:
# Location Code
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.countplot(df['Location Code'])

plt.subplot(1, 2, 2)
sns.barplot(x='Location Code', y='Customer Lifetime Value', data=df)

In [ ]:
# Marital Status
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.countplot(df['Marital Status'])

plt.subplot(1, 2, 2)
sns.barplot(x='Marital Status', y='Customer Lifetime Value', data=df)

In [ ]:
# Policy Type
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.countplot(df['Policy Type'])

plt.subplot(1, 2, 2)
sns.barplot(x='Policy Type', y='Customer Lifetime Value', data=df)

In [ ]:
# Policy

df['Policy'].unique()

In [ ]:
df['Policy'] = df['Policy'].factorize()[0]  # 보험의 9가지 범주형 데이터 값을 숫자로 바꿈

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.countplot(df['Policy'])

plt.subplot(1, 2, 2)
sns.barplot(x='Policy', y='Customer Lifetime Value', data=df)

In [ ]:
# Sales Channel
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.countplot(df['Sales Channel'])

plt.subplot(1, 2, 2)
sns.barplot(x='Sales Channel', y='Customer Lifetime Value', data=df)

In [ ]:
# Renew Offer Type
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.countplot(df['Renew Offer Type'])

plt.subplot(1, 2, 2)
sns.barplot(x='Renew Offer Type', y='Customer Lifetime Value', data=df)

In [ ]:
# Vehicle Class
plt.figure(figsize=(28,6))
plt.subplot(1, 2, 1)
sns.countplot(df['Vehicle Class'])

plt.subplot(1, 2, 2)
sns.barplot(x='Vehicle Class', y='Customer Lifetime Value', data=df)

In [ ]:
# Vehicle Size
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
sns.countplot(df['Vehicle Size'])

plt.subplot(1, 2, 2)
sns.barplot(x='Vehicle Size', y='Customer Lifetime Value', data=df)

대부분의 16개 범주형 데이터는 LTV와 뚜렷한 상관관계가 보이지 않는다.

Coverage의 경우 Premium > Extended Premium > Basic 순으로 LTV가 높다.

Renew Offer Type은 offer 1 > offer 3 > offer 2 > offer 4 순으로 LTV가 다소 차이를 보인다. 

Vehicle Class의 경우 럭셔리 suv와 럭셔리 카가 상대적으로 LTV가 높다. 

# 3. Feature Engineering

In [ ]:
# drop insignificant columns
df = df.drop(['Income', 'Months Since Last Claim', 'Months Since Policy Inception','Customer','State','Response','Education','Effective To Date', 'Month','EmploymentStatus','Gender','Location Code','Marital Status','Policy Type','Policy','Sales Channel','Vehicle Size'], axis=1)

LTV와 상관관계가 없다고 생각하는 16개의 컬럼 드롭

In [ ]:
df.head()

In [ ]:
# 텍스트 컬럼 숫자로 바꾸기
df['Coverage'] = df['Coverage'].factorize()[0] 

In [ ]:
df['Renew Offer Type'] = 0
df.loc[(df['Renew Offer Type'] == 'Offer1'), 'Renew Offer Type'] = 0
df.loc[(df['Renew Offer Type'] == 'Offer2'), 'Renew Offer Type'] = 1
df.loc[(df['Renew Offer Type'] == 'Offer3'), 'Renew Offer Type'] = 2
df.loc[(df['Renew Offer Type'] == 'Offer4'), 'Renew Offer Type'] = 3

In [ ]:
df['Vehicle Class'] = 0
df.loc[(df['Vehicle Class'] == 'Two-Door Car'), 'Vehicle Class'] = 0
df.loc[(df['Vehicle Class'] == 'Four-Door Car'), 'Vehicle Class'] = 1
df.loc[(df['Vehicle Class'] == 'SUV'), 'Vehicle Class'] = 2
df.loc[(df['Vehicle Class'] == 'Luxury SUV'), 'Vehicle Class'] = 3
df.loc[(df['Vehicle Class'] == 'Sports Car'), 'Vehicle Class'] = 4
df.loc[(df['Vehicle Class'] == 'Luxury Car'), 'Vehicle Class'] = 5

In [ ]:
df.head()

In [ ]:
# prediction target
y = df['Customer Lifetime Value']

In [ ]:
df_features = ['Coverage', 'Monthly Premium Auto', 'Number of Policies', 'Renew Offer Type', 'Total Claim Amount', 'Vehicle Class', 'Number of Open Complaints']

In [ ]:
X = df[df_features]

In [ ]:
X.head()

# 4. Modeling 

In [ ]:
# 사이킷런 및 관련 알고리즘 불러오기
from sklearn.model_selection import train_test_split  # 훈련 및 데스트용 데이터 분할 모듈 불러오기
from sklearn.tree import DecisionTreeRegressor #의사결정나무 회귀
from sklearn.ensemble import RandomForestRegressor #랜럼포레스트 회귀
from xgboost import XGBRegressor #XGB 회귀
from sklearn.metrics import accuracy_score #평가 모듈 불러오기

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state = 0)

**1) DecisionTreeRegressor**

In [ ]:
# 맥스 리프노드 설정에 따라 의사결정나무 결과를 출력하는 함수 작성
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    val_pred = model.predict(val_X)
    mae = mean_absolute_error(val_y, val_pred)
    return(mae)

from sklearn.metrics import mean_absolute_error # 결과를 평가하기 위한 모듈 mae 불러오기

candidate_max_leaf_nodes = [25, 50, 100, 250, 500, 1000, 2000]
for max_leaf_nodes in candidate_max_leaf_nodes:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Accuracy Score:  %d" %(max_leaf_nodes, my_mae))

최대 리프노드의 수(max_leaf_nodes)가 2000개일 때 예측값과 테스트 데이터 y값의 절대값 오차(mae, mean_absolute_error)가 가장 작음

**2) RandomForestRegressor**

In [ ]:
# n_estimators에 따라 랜덤포레스트 적용 결과를 출력하는 함수 작성
def get_mae(n_estimators, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

candidate_n_estimators = [25, 50, 100, 250, 500, 1000]
for n_estimators in candidate_n_estimators:
    my_mae = get_mae(n_estimators, train_X, val_X, train_y, val_y)
    print("n_estimators: %d  \t\t Mean Absolute Error:  %d" %(n_estimators, my_mae))

n_estimators(의사결정 나무개수)가 500개일 때 mae가 가장 작음

**3) XGBRegressor**

In [ ]:
# n_estimators에 따라 XGBRegressor의 결과를 출력하는 함수 작성
def get_mae(n_estimators, train_X, val_X, train_y, val_y):
    model = XGBRegressor(n_estimators=n_estimators, learning_rate=0.05)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

candidate_n_estimators = [100, 250, 500, 1000]
for n_estimators in candidate_n_estimators:
    my_mae = get_mae(n_estimators, train_X, val_X, train_y, val_y)
    print("n_estimators: %d  \t\t Mean Absolute Error:  %d" %(n_estimators, my_mae))

n_estimators가 100개 때 mae가 가장 작음

In [ ]:
# Best model
best_model = RandomForestRegressor(n_estimators=500, random_state=0)
best_model.fit(train_X, train_y)
preds_val = best_model.predict(val_X)
mae = mean_absolute_error(val_y, preds_val)
print("베스트 모델을 적용한 결과 mae는 %d로 가장 낮았다." %mae)

LTV 평균이 약 8,004달러임을 감안할 때 1,518의 오차는 약 78%의 정확도를 말한다. 

In [ ]:
df_1 = pd.DataFrame({'Actual': val_y, 'Predicted': preds_val})
df_1

In [ ]:
# val_y과 preds_val 그래프 그리기
from matplotlib.pyplot import figure
figure(num=None, figsize=(20,6), dpi=80)

plt.subplot(1, 2, 1)
x1= np.arange(1827)
plt.plot(x1, df_1['Actual'] , label='Actual LTV')
plt.plot(x1, df_1['Predicted'], label='Predicted LTV') 
plt.legend()

plt.subplot(1, 2, 2)
sns.scatterplot(x='Actual', y= 'Predicted', data=df_1) 

# 5. Conclusion

LTV는 Upper fence(Q3 + 1.5 * IQR)=16414를 벗어나는 이상치 데이터가 819개로 이것이 테스트 데이터 예측값과 실제값의 차이를 만드는 주요 원인으로 보인다. 피처 엔지니어링이나 다른 알고리즘을 적용해보는 것과 함께 이상치를 어떻게 처리할 지가 예측 정확도를 개선할 수 있는 주요 방법이 될 것 같다. 